This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/simple_legs.html).  I recommend having both windows open, side-by-side!


In [ ]:
import mpld3
from IPython.display import display
from matplotlib import pyplot as plt
from pydrake.all import (
    ConstantVectorSource,
    DiagramBuilder,
    LogVectorOutput,
    MeshcatVisualizer,
    SceneGraph,
    Simulator,
    StartMeshcat,
)
from pydrake.examples import CompassGait, CompassGaitGeometry

from underactuated import running_as_notebook


if running_as_notebook:
    mpld3.enable_notebook()

In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

# The Compass Gait

In [ ]:
def compass_gait():
    builder = DiagramBuilder()
    compass_gait = builder.AddSystem(CompassGait())

    hip_torque = builder.AddSystem(ConstantVectorSource([0.0]))
    builder.Connect(
        hip_torque.get_output_port(0), compass_gait.get_input_port(0)
    )

    scene_graph = builder.AddSystem(SceneGraph())
    CompassGaitGeometry.AddToBuilder(
        builder,
        compass_gait.get_floating_base_state_output_port(),
        scene_graph,
    )
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    meshcat.Set2dRenderMode(xmin=-1, xmax=5, ymin=-1, ymax=2)

    logger = LogVectorOutput(compass_gait.get_output_port(1), builder)

    diagram = builder.Build()
    simulator = Simulator(diagram)

    context = simulator.get_mutable_context()
    context.SetAccuracy(1e-4)
    context.SetContinuousState([0.0, 0.0, 0.4, -2.0])

    visualizer.StartRecording(False)
    simulator.AdvanceTo(8.0)
    visualizer.PublishRecording()
    log = logger.FindLog(context)
    plt.figure()
    plt.plot(log.data()[4, :], log.data()[11, :])
    plt.xlabel("left leg angle")
    plt.ylabel("left leg angular velocity")
    display(mpld3.display())


compass_gait()